<a href="https://colab.research.google.com/github/dini-5002/Grammar_check/blob/main/FineTuned1_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Installing required modules

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

In [ ]:
!pip install datasets

In [ ]:
 !pip install dataset transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 128.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.41
    Uninstalling SQLAlchemy-2.0.41:
      Successfully uninstalled SQLAlchemy-2.0.41
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4
ERROR: pip's dependency resolver does not currently take into account all the package

In [ ]:
!pip install wandb

In [ ]:
!pip install transformers[torch]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

##Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import torch

In [ ]:
from transformers import (AdamW,
    T5ForConditionalGeneration, T5Tokenizer,
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )


ImportError: cannot import name 'AdamW' from 'transformers' (/usr/local/lib/python3.11/dist-packages/transformers/__init__.py)

#Loading model and finetuning on the required dataset

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('google-t5/t5-small')
model = T5ForConditionalGeneration.from_pretrained('google-t5/t5-small')

In [ ]:
df = pd.read_csv('/content/C4200M_50k.csv')
df.head()

,Unnamed: 0,input,label
0,0,The effect of widespread dud targets two face ...,"1. The effect of ""widespread dud"" targets two ..."
1,1,tax on sales of stores for non residents are s...,Capital Gains tax on the sale of properties fo...
2,2,Much many brands and sellers still in the market.,Many brands and sellers still in the market.
3,3,this is is the latest Maintenance release of S...,This is is the latest maintenance release of S...
4,4,"Fairy Or Not, I'm the Godmother: no just look,...","Fairy Or Not, I'm the Godmother: Not just a lo..."


In [ ]:
#split it into training and evaluation
from sklearn.model_selection import train_test_split
train_df, eval_df = train_test_split(df, test_size=0.1, random_state=42)

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(eval_df)

In [ ]:
from torch.utils.data import Dataset, DataLoader
class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer, print_text= False, trucation = True):
        self.dataset = dataset
        self.pad_to_max_length = True
        self.tokenizer = tokenizer
        self.print_text = print_text
        self.max_len = 64

    def __len__(self):
        return len(self.dataset)

    def tokenize_data(self, example):
      input_, target_ = example['input'], example['label']

      #tokenize inputs
      tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length, max_length = self.max_len,
                                   return_attention_mask=True, truncation=True)
      tokenized_targets = tokenizer(target_, pad_to_max_length = self.pad_to_max_length, max_length=self.max_len, return_attention_mask=True,truncation=True)
      inputs = {"input_ids": tokenized_inputs['input_ids'],
                "attention_mask": tokenized_inputs['attention_mask'],
                "labels": tokenized_targets['input_ids']}
      return inputs

    def  __getitem__(self, index):
       inputs = self.tokenize_data(self.dataset[index])
       return inputs

In [ ]:
datacollater = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
args = Seq2SeqTrainingArguments(output_dir='./results',
                                evaluation_strategy="epoch",
                                learning_rate=2e-5,
                                per_device_train_batch_size=16,
                                per_device_eval_batch_size=16,
                                fp16=True,
)
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    tokenizer = tokenizer,
    train_dataset=GrammarDataset(train_dataset, tokenizer),
    eval_dataset=GrammarDataset(test_dataset, tokenizer),
    data_collator=datacollater,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.425200,0.380454
2,0.405700,0.369074


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
torch.save(model.state_dict(), '/content/gdrive/MyDrive/modelT5f4.pt')

In [ ]:
mod=T5ForConditionalGeneration.from_pretrained('google-t5/t5-small')

In [ ]:
mod.load_state_dict(torch.load('/content/drive/MyDrive/modelT5f4.pt'))

In [ ]:
def correct_text(text):
    inputs = tokenizer(text, return_tensors="pt")
    output = mod.generate(**inputs)
    corrected_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return corrected_text


In [ ]:
sentence = "what up"

In [ ]:
correct_text(sentence)

'what up?'

In [ ]:
#split it into training and evaluation
from sklearn.model_selection import train_test_split
train_df, eval_df = train_test_split(df, test_size=0.1, random_state=42)

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(eval_df)

In [ ]:
!pip install evaluate



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.0 MB/s eta 0:00:00


In [ ]:
from evaluate import load
import numpy as np


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=9c15f85ae8463fb8c83977aaee1dfdd54330de1670260afec08903e4db6f6c36
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
rouge = load("rouge")  # or use "bleu"



In [ ]:
def evaluate_model(model, tokenizer, dataset):
    predictions = []
    references = []

    for example in tqdm(dataset, desc="Evaluating"):
        input_text = example["input"]
        reference = example["label"]

        input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True)
        output_ids = model.generate(input_ids, max_length=64, num_beams=4)
        prediction = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        predictions.append(prediction)
        references.append(reference)

    results = rouge.compute(predictions=predictions, references=references)
    return results


In [ ]:

from tqdm.notebook import tqdm


In [ ]:
results = evaluate_model(mod, tokenizer, test_dataset)
print("Evaluation Scores:", results)


Evaluating:   0%|          | 0/5000 [00:00<?, ?it/s]

Evaluation Scores: {'rouge1': np.float64(0.8437526987128772), 'rouge2': np.float64(0.7151004444512036), 'rougeL': np.float64(0.8370021993052528), 'rougeLsum': np.float64(0.8369393214595818)}
